## Imports

In [1]:
import os
import sys
import os
import glob
import random
import time
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Dataset

import torchvision.models as models
from torchvision.models import VGG16_Weights

import torchvision.transforms.functional as F

from torchvision.transforms.functional import to_tensor, resize, center_crop, rotate, adjust_brightness
from torchvision.transforms.functional import resize

## Constants

## constants

In [2]:
DATA_PATH = "/kaggle/input/lgg-mri-segmentation/kaggle_3m/"

plt.style.use("dark_background")

IMG_SIZE = 512
MASK_SIZE = 128

EPOCHS = 150
PATIENCE = 5 # Stop training if no improvement for 'patience' epochs
BATCH_SIZE = 8

## Constants

## Utility Functions

In [3]:
def read_data_pathes(path):
    data_map = []
    for sub_dir_path in glob.glob(path+"*"):
        if not os.path.isdir(sub_dir_path):
            print("This is not a dir:", sub_dir_path)
            continue
        dirname = sub_dir_path.split("/")[-1]
        for filename in os.listdir(sub_dir_path):
            image_path = sub_dir_path + "/" + filename
            data_map.extend([dirname, image_path])
        
    df = pd.DataFrame({'dirname' : data_map[::2], 'path' : data_map[1::2]})
    return df

df = read_data_pathes(DATA_PATH)
df.head(5)

def separate_and_sort_paths(df, base_len=89, end_img_len=4, end_mask_len=9):
    df_imgs = df[~df['path'].str.contains("mask")]
    df_masks = df[df['path'].str.contains("mask")]

    imgs = sorted(df_imgs["path"].values, key=lambda x: int(x[base_len:-end_img_len]))
    masks = sorted(df_masks["path"].values, key=lambda x: int(x[base_len:-end_mask_len]))

    df = pd.DataFrame({'patient': df_imgs.dirname.values, 'image_path': imgs, 'mask_path': masks})

    return imgs, masks, df

def prepare_sample_images(df, row_count, sample_pos_df, sample_neg_df):
    sample_pos_df = sample_pos_df['image_path'].values
    sample_neg_df = sample_neg_df['image_path'].values

    sample_imgs = []
    for neg, pos in zip(sample_neg_df, sample_pos_df):
        neg_img = cv2.resize(cv2.imread(neg), (IMG_SIZE, IMG_SIZE))
        pos_img = cv2.resize(cv2.imread(pos), (IMG_SIZE, IMG_SIZE))
        sample_imgs.extend([pos_img, neg_img])

    sample_pos_arr = np.vstack(sample_imgs[::2])
    sample_neg_arr = np.vstack(sample_imgs[1::2])

    return sample_pos_arr, sample_neg_arr

def upsample_dataset(initial_df):
    pos_counts = initial_df['diagnosis'].value_counts() 
    max_count = pos_counts.max()

    dfs = []
    for label, count in pos_counts.items():
        subset_df = initial_df[initial_df['diagnosis'] == label]
        if count < max_count:
            repeat_count = max_count - count
            repeated_subset_df = subset_df.sample(repeat_count, replace=True)
            dfs.append(repeated_subset_df)
    
    return pd.concat([initial_df] + dfs, ignore_index=True).sample(frac=1).reset_index(drop=True)

This is not a dir: /kaggle/input/lgg-mri-segmentation/kaggle_3m/README.md
This is not a dir: /kaggle/input/lgg-mri-segmentation/kaggle_3m/data.csv


In [4]:
def positiv_negativ_diagnosis(mask_path):
    value = np.max(cv2.imread(mask_path))
    if value > 0 : return 1
    else: return 0

def get_sample_df(df, type, expected, count, column=None):
    sample_df = df[df[type] == expected].sample(count)
    if column: return sample_df[column]
    return sample_df

def get_df_sample(df, type, expected, count):
    return get_sample_df(df, type, expected, count).values

def get_img_sample(df, type, expected, count):
    return get_sample_df(df, type, expected, count, column='image_path').values
    
def resize(img, size):
    return cv2.resize(cv2.imread(img), size)

def split(df, test_size):
    df1, df2 = train_test_split(df, stratify=df.diagnosis, test_size=test_size)
    return df1.reset_index(drop=True), df2.reset_index(drop=True) # Is this nessesary or we can del it, for cheating

def print_data_shapes(train_df, val_df, test_df):
    print(f"{'Dataset':<10} {'Shape':<15}")
    print(f"{'-'*25}")
    print(f"{'Train':<10} {train_df.shape}")
    print(f"{'Validation':<10} {val_df.shape}")
    print(f"{'Test':<10} {test_df.shape}")

## Plot Functions

In [5]:
def plot_histogram(df, ylabel='Image Count', title='Dataset Histogram'): 
    ax = df.diagnosis.value_counts().plot(kind='bar', stacked=True, figsize=(10, 6), color=selected_colors)

    for i, rows in enumerate(df.diagnosis.value_counts().values):
        ax.annotate(int(rows), 
                    xy=(i, rows-12), rotation=0, color='white', ha='center', verticalalignment='bottom', fontsize=15, fontweight='bold')

    bounding_box = dict(boxstyle='round', fc=('lightblue'), ec=('black'))
    ax.text(1.2, 2550, f"Total {len(df)} images", size=15, color='black', ha='center', va='center', bbox=bounding_box)
    ax.set_xticklabels(labels, rotation=45, fontsize=12);
    ax.set_ylabel(ylabel, fontsize=12)
    ax.set_title(title,fontsize = 18, y=1.05)

def plot_distribution(df, ylabel='Total Images', title='Dataset Distribution'):
    patients_by_diagnosis = df.groupby(['patient', 'diagnosis'])['diagnosis'].size().unstack().fillna(0)
    patients_by_diagnosis.columns = ["Positive", "Negative"]
    
    ax = patients_by_diagnosis.plot(kind='bar',stacked=True, figsize=(18, 10), color=selected_colors, alpha=0.9)
    ax.legend(fontsize=20);
    ax.set_xlabel('Patients',fontsize = 20)
    ax.set_ylabel(ylabel, fontsize = 20)
    ax.set_title(title,fontsize = 25, y=1.005)
    ax.set_xticklabels([])

def plot_masks(df):
    sample_df = sample_pos_df.values
    sample_imgs = []
    for i, data in enumerate(sample_df):
        img = resize(data[1], (IMG_SIZE, IMG_SIZE))
        mask = resize(data[2], (MASK_SIZE, MASK_SIZE))
        sample_imgs.extend([img, mask])
    
    sample_imgs_arr = np.hstack(np.array(sample_imgs[::2]))
    sample_masks_arr = np.hstack(np.array(sample_imgs[1::2]))
    
    fig = plt.figure(figsize=(10, 10))
    grid = ImageGrid(fig, 111, nrows_ncols=(2, 1), axes_pad=0.1)
    
    grid[0].imshow(sample_imgs_arr)
    grid[0].set_title("Images", fontsize=15)
    grid[0].axis("off")
    grid[1].imshow(sample_masks_arr)
    grid[1].set_title("Masks", fontsize=15, y=0.9)
    grid[1].axis("off")
    plt.show()

def plot_aug(data, mask=False):
    ncols, nrows = len(data), 1
    size = MASK_SIZE if mask else IMG_SIZE
    plt.figure(figsize=(ncols * (size / 100), nrows * (size / 100)))
    plt.subplots_adjust(wspace=0., hspace=0.)
    idx = 0
    for img in data:
        if mask:
            img = img.numpy().astype(np.float32)[0, :, :]
        else:
            img = img.numpy().transpose(1, 2, 0)

        plt.subplot(nrows, ncols, idx + 1)
        plt.imshow(img)
        plt.axis('off')
        idx += 1

    return plt.show()

def plot_metrics(train_losses, val_losses, train_dice_scores, val_dice_scores):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'r', label='Train Loss')
    plt.plot(epochs, val_losses, 'b', label='Val Loss')
    plt.title('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_dice_scores, 'r', label='Train Dice')
    plt.plot(epochs, val_dice_scores, 'b', label='Val Dice')
    plt.title('Dice Coefficient')
    plt.legend()

    plt.show()

## Metric Functions

In [6]:
def dice_coefficient(output, target, smooth=1e-6):
    output = (output > 0.5).float()
    intersection = (output * target).sum()
    return (2.0 * intersection + smooth) / (output.sum() + target.sum() + smooth)
# # def dice_coef_metric(inputs, target):
#     intersection = 2.0 * (target * output).sum()
#     union = target.sum() + output.sum()
#     if target.sum() == 0 and output.sum() == 0:
#         return 1.0

#     return intersection / union

def iou_score(output, target, smooth=1e-6):
    output = (output > 0.5).float()
    intersection = (output * target).sum()
    union = output.sum() + target.sum() - intersection
    return (intersection + smooth) / (union + smooth)

def dice_loss(output, target, smooth=1e-6):
    return 1 - dice_coefficient(output, target)
    # intersection = (output * target).sum()
    # return 1 - (2.0 * intersection + smooth) / (output.sum() + target.sum() + smooth)

    # smooth = 1.0
    # intersection = 2.0 * ((target * output).sum()) + smooth
    # union = target.sum() + output.sum() + smooth

    # return 1 - (intersection / union)

## Constants

In [7]:
df = read_data_pathes(DATA_PATH)
df.head(5)

This is not a dir: /kaggle/input/lgg-mri-segmentation/kaggle_3m/README.md
This is not a dir: /kaggle/input/lgg-mri-segmentation/kaggle_3m/data.csv


,dirname,path
0,TCGA_DU_7010_19860307,/kaggle/input/lgg-mri-segmentation/kaggle_3m/T...
1,TCGA_DU_7010_19860307,/kaggle/input/lgg-mri-segmentation/kaggle_3m/T...
2,TCGA_DU_7010_19860307,/kaggle/input/lgg-mri-segmentation/kaggle_3m/T...
3,TCGA_DU_7010_19860307,/kaggle/input/lgg-mri-segmentation/kaggle_3m/T...
4,TCGA_DU_7010_19860307,/kaggle/input/lgg-mri-segmentation/kaggle_3m/T...


In [8]:
DATA_PATH = "/kaggle/input/lgg-mri-segmentation/kaggle_3m/"

plt.style.use("dark_background")

In [9]:
imgs, masks, df = separate_and_sort_paths(df)

In [ ]:
df['diagnosis'] = df["mask_path"].apply(lambda m: positiv_negativ_diagnosis(m))
# df.head()

In [ ]:
positive_color = 'lightseagreen' 
negative_color = 'violet'
labels = ['Positive', 'Negative']
selected_colors = [positive_color, negative_color]
colors = ['mediumvioletred', 'springgreen']
sample_pos_df = get_sample_df(df, 'diagnosis', expected=1, count=3)
sample_neg_df = get_sample_df(df, 'diagnosis', expected=0, count=3)

In [ ]:
plot_histogram(df)
plot_distribution(df)
# plot_masks(df)

In [ ]:
# sample_yes_arr, sample_no_arr = prepare_sample_images(df, row_count=3, sample_pos_df, sample_neg_df)
# # Plot
# fig = plt.figure(figsize=(10, 10))
# grid = ImageGrid(fig, 111, nrows_ncols=(1, 4), axes_pad=0.1)

# grid[0].imshow(sample_yes_arr)
# grid[0].set_title("Positive")
# grid[0].axis("off")

# grid[1].imshow(sample_no_arr)
# grid[1].set_title("Negative")
# grid[1].axis("off")

# grid[2].imshow(sample_yes_arr[:, :, 0], cmap="hot")
# grid[2].set_title("Positive")
# grid[2].axis("off")

# grid[3].imshow(sample_no_arr[:, :, 0], cmap="hot")
# grid[3].set_title("Negative")
# grid[3].axis("off")

# plt.figtext(0.36, 0.90, "Original", va="center", ha="center", size=15)
# plt.figtext(0.66, 0.90, "With hot colormap", va="center", ha="center", size=15)
# plt.show()


In [ ]:
df = upsample_dataset(df)
plot_histogram(df)

In [ ]:
train_df, temp_df = split(df, test_size=0.2)
test_df, val_df = split(temp_df, test_size=0.5)

print_data_shapes(train_df, val_df, test_df)

## Model

In [ ]:
class AugmentPair:
    def __init__(self, rotation_range=15, scale_range=0.2, brightness_range=(0.9, 1.1)):
        self.rotation_range = rotation_range
        self.scale_range = scale_range
        self.brightness_range = brightness_range

    def rotate(self, input, angle):
        return rotate(input, angle)

    def scale(self, data, factor):
        width, height = data.size
        data = resize(data, (int(height * factor), int(width * factor)))
        return center_crop(data, (height, width))
    
    def adjust_brightness(self, input, factor):
        return adjust_brightness(input, factor)        
        
    def __call__(self, pair):
        image, mask = pair

        angle = np.random.uniform(-self.rotation_range, self.rotation_range)
        image = self.rotate(image, angle)
        mask = self.rotate(mask, angle)

        factor = np.random.uniform(1 - self.scale_range, 1 + self.scale_range)
        image = self.scale(image, factor)
        mask = self.scale(mask, factor)

        image = self.adjust_brightness(image, factor=np.random.uniform(*self.brightness_range))

        return image, mask

class MRIDataset(Dataset):
    def __init__(self, df, transformer=None):
        self.df = df
        self.transformer = transformer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Load image and mask with OpenCV
        image = cv2.imread(self.df.iloc[idx, 1])
        mask = cv2.imread(self.df.iloc[idx, 2], 0)  # 0 for grayscale

        # Convert to PIL Image
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        mask = Image.fromarray(mask)

        # Apply transformations if provided
        real_image = image
        real_mask = mask
        if self.transformer:
            image, mask = self.transformer((image, mask))

        # Convert to tensors
        real_image = to_tensor(real_image)
        real_mask = torch.tensor(np.array(mask) / 255.0, dtype=torch.float32).unsqueeze(0)
        image = to_tensor(image)
        mask = torch.tensor(np.array(mask) / 255.0, dtype=torch.float32).unsqueeze(0)

        return image, resize(mask, MASK_SIZE)


train_dataset = MRIDataset(train_df, transformer=AugmentPair())
train_dataloader = DataLoader(train_dataset, batch_size=32, num_workers=4, shuffle=True)

val_dataset = MRIDataset(val_df, transformer=None)
val_dataloader = DataLoader(val_dataset, batch_size=32, num_workers=4, shuffle=False)

test_dataset = MRIDataset(test_df, transformer=None)
test_dataloader = DataLoader(test_dataset, batch_size=32, num_workers=4, shuffle=False)

images, masks = next(iter(train_dataloader))
print(images.shape, masks.shape)

In [ ]:
# combined_df = pd.concat([sample_pos_df, sample_neg_df], ignore_index=True)
# shuffled_df = combined_df.sample(frac=1).reset_index(drop=True)
# dataloader_ = DataLoader(MRIDataset(shuffled_df, transformer=AugmentPair()), 
#                          batch_size=32, num_workers=4, shuffle=True)

# images_, masks_, real_images_, real_masks_ = next(iter(dataloader_))
# plot_aug(images_)
# plot_aug(masks_)
# plot_aug(real_images_)
# plot_aug(real_masks_)

In [ ]:
class VGGEncoder(nn.Module):
    def __init__(self, pretrained=True):
        super(VGGEncoder, self).__init__()
        weights = VGG16_Weights.IMAGENET1K_V1 if pretrained else None
        vgg = models.vgg16(weights=weights)
        features = list(vgg.features.children())
        # Extract blocks for skip connections
        self.block1 = nn.Sequential(*features[:5])   # block1_pool
        self.block2 = nn.Sequential(*features[5:10]) # block2_pool
        self.block3 = nn.Sequential(*features[10:17])# block3_pool
        self.block4 = nn.Sequential(*features[17:24])# block4_pool
        self.block5 = nn.Sequential(*features[24:31])# block5_pool

    def forward(self, x):
        skip_connections = []
        x = self.block1(x)
        # print('blck1:', x.shape)
        skip_connections.append(x)
        x = self.block2(x)
        # print('blck2:', x.shape)
        skip_connections.append(x)
        x = self.block3(x)
        # print('blck3:', x.shape)
        skip_connections.append(x)
        x = self.block4(x)
        # print('blck4:', x.shape)
        skip_connections.append(x)
        x = self.block5(x)
        # print('blck5:', x.shape)
        # print()
        # print('VGG output shape:', x.shape)
        return x, skip_connections[::-1]  # Reverse for decoding
        
class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super(DecoderBlock, self).__init__()
        self.upsample = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)
        self.conv1 = nn.Conv2d(out_channels + skip_channels, out_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x, skip):
        x = self.upsample(x)  # Upsample
        x = torch.cat([x, skip], dim=1)  # Concatenate with skip connection
        x = self.relu(self.conv1(x))  # First convolution
        x = self.relu(self.conv2(x))  # Second convolution
        return x

class UNetDecoder(nn.Module):
    def __init__(self, num_classes=1):
        super(UNetDecoder, self).__init__()
        self.decoder4 = DecoderBlock(512, 512, 256)  # Block 5 to Block 4
        self.decoder3 = DecoderBlock(256, 256, 128)  # Block 4 to Block 3
        self.decoder2 = DecoderBlock(128, 128, 64)   # Block 3 to Block 2
        self.decoder1 = DecoderBlock(64, 64, 32)     # Block 2 to Block 1

        # Final output layer
        self.final_conv = nn.Conv2d(32, num_classes, kernel_size=1)

    def forward(self, x, skips):
        x = self.decoder4(x, skips[0])  # Skip from Block 4
        x = self.decoder3(x, skips[1])  # Skip from Block 3
        x = self.decoder2(x, skips[2])  # Skip from Block 2
        x = self.decoder1(x, skips[3])  # Skip from Block 1
        x = self.final_conv(x)          # Output layer
        return x

class UNet(nn.Module):
    def __init__(self, num_classes=1):
        super(UNet, self).__init__()
        self.encoder = VGGEncoder()
        self.decoder = UNetDecoder()

    def forward(self, x):
        x, skips = self.encoder(x)  # Encoder forward pass
        x = self.decoder(x, skips)  # Decoder forward pass
        return x

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss, total_dice, total_iou = 0, 0, 0

    for images, masks in tqdm(dataloader, desc="Training", leave=False):    
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = dice_loss(outputs, masks) + criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_dice += dice_coefficient(outputs, masks).item()
        total_iou += iou_score(outputs, masks).item()

    return (total_loss / len(dataloader)), (total_dice / len(dataloader)), (total_iou / len(dataloader))

def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()
    total_loss, total_dice, total_iou = 0, 0, 0

    with torch.no_grad():
        for images, masks in tqdm(dataloader, desc="Validating", leave=False):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = dice_loss(outputs, masks) + criterion(outputs, masks)
            total_loss += loss.item()
            total_dice += dice_coefficient(outputs, masks).item()
            total_iou += iou_score(outputs, masks).item()

    return total_loss / len(dataloader), total_dice / len(dataloader), total_iou / len(dataloader)


In [ ]:
def test_model(model, dataloader, device):
    model.eval()
    total_dice, total_iou = 0, 0
    samples = []

    with torch.no_grad():
        for images, masks in dataloader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)

            total_dice += dice_coefficient(outputs, masks).item()
            total_iou += iou_score(outputs, masks).item()

            # Store sample predictions for visualization
            samples.append((images.cpu(), masks.cpu(), outputs.cpu()))

    avg_dice = total_dice / len(dataloader)
    avg_iou = total_iou / len(dataloader)
    
    return avg_dice, avg_iou, samples


In [ ]:
model = UNet()
print(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=3e-4)

In [ ]:
early_stop_counter = 0
best_val_loss = np.inf

train_losses, val_losses = [], []
train_dice_scores, val_dice_scores = [], []

for epoch in range(EPOCHS):
    train_loss, train_dice, train_iou = train_one_epoch(model, train_dataloader, criterion, optimizer, device)
    val_loss, val_dice, val_iou = validate_one_epoch(model, val_dataloader, criterion, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_dice_scores.append(train_dice)
    val_dice_scores.append(val_dice)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Dice: {train_dice:.4f}, Train IoU: {train_iou:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Dice: {val_dice:.4f}, Val IoU: {val_iou:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0 
        torch.save(model.state_dict(), "best_model.pth")
    else:
        early_stop_counter += 1
        if early_stop_counter >= PATIENCE:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

print("Training complete.")


In [ ]:
plot_metrics(train_losses, val_losses, train_dice_scores, val_dice_scores)

In [ ]:
state_dict = torch.load("best_model.pth", map_location=device)
model.load_state_dict(state_dict)
model.to(device) 

test_loss, test_dice, test_iou = validate_one_epoch(model, test_dataloader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Dice: {test_dice:.4f}, Test IoU: {test_iou:.4f}")

In [ ]:
# # image
# test_sample = test_df[test_df["diagnosis"] == 1].sample(1).values[0]
# image = cv2.resize(cv2.imread(test_sample[1]), (128, 128))

# #mask
# mask = cv2.resize(cv2.imread(test_sample[2]), (128, 128))

# pred = model.forward(image)



# # pred
# pred = torch.tensor(image.astype(np.float32) / 255.).unsqueeze(0).permute(0,3,1,2)
# pred = model(pred.to(device))
# pred = validate_one_epoch(model, test_dataloader, criterion, device)
# pred = pred.detach().cpu().numpy()[0,0,:,:]

# # # pred with tshd
# # pred_t = np.copy(pred)
# # pred_t[np.nonzero(pred_t < 0.3)] = 0.0
# # pred_t[np.nonzero(pred_t >= 0.3)] = 255.#1.0
# # pred_t = pred_t.astype("uint8")

# # plot
# fig, ax = plt.subplots(nrows=2,  ncols=2, figsize=(10, 10))

# ax[0, 0].imshow(image)
# ax[0, 0].set_title("image")
# ax[0, 1].imshow(mask)
# ax[0, 1].set_title("mask")
# ax[1, 0].imshow(pred)
# ax[1, 0].set_title("prediction")
# ax[1, 1].imshow(pred_t)
# ax[1, 1].set_title("prediction with threshold")
# plt.show()